In [1]:
import pandas as pd

In [2]:
raw_df = pd.read_csv('./통근 인구.csv', encoding='euc-kr', header=1)
print(raw_df.head(3))

  행정구역별(시군구) 통근·통학여부별   소요시간별  통근·통학 인구(12세 이상)   단일 수단-계      걸어서     자전거  \
0         전국        계       계          28012287  26470392  5122567  372082   
1         전국        계  15분 미만           6536753   6536745  3024510  138508   
2         전국        계   15~30           7803902   7574019  1658932  153724   

  승용차승합차(벤)      트럭 시내좌석마을버스   통근통학차량  고속시외버스    전철지하철     기차     택시      기타  \
0  12228703  628472  3750677  1243965  116473  2645111  24902  74561  262879   
1   2580349  214009   237287   133074       -    42562      -  31692  134754   
2   3707902  152403  1060547   503199       -   231325     14  29826   76147   

      복합수단  
0  1541895  
1        8  
2   229883  


In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   행정구역별(시군구)        501 non-null    object
 1   통근·통학여부별          501 non-null    object
 2   소요시간별             501 non-null    object
 3   통근·통학 인구(12세 이상)  501 non-null    int64 
 4   단일 수단-계           501 non-null    object
 5   걸어서               501 non-null    object
 6   자전거               501 non-null    object
 7   승용차승합차(벤)         501 non-null    object
 8   트럭                501 non-null    object
 9   시내좌석마을버스          501 non-null    object
 10  통근통학차량            501 non-null    object
 11  고속시외버스            501 non-null    object
 12  전철지하철             501 non-null    object
 13  기차                501 non-null    object
 14  택시                501 non-null    object
 15  기타                501 non-null    object
 16  복합수단              501 non-null    object
dtypes: int64(1), obj

In [4]:
# Replace '-' with 0 and convert columns to integers
df = raw_df.replace('-', 0)
for column in df.columns[5:]:
    df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).astype(int)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   행정구역별(시군구)        501 non-null    object
 1   통근·통학여부별          501 non-null    object
 2   소요시간별             501 non-null    object
 3   통근·통학 인구(12세 이상)  501 non-null    int64 
 4   단일 수단-계           501 non-null    object
 5   걸어서               501 non-null    int32 
 6   자전거               501 non-null    int32 
 7   승용차승합차(벤)         501 non-null    int32 
 8   트럭                501 non-null    int32 
 9   시내좌석마을버스          501 non-null    int32 
 10  통근통학차량            501 non-null    int32 
 11  고속시외버스            501 non-null    int32 
 12  전철지하철             501 non-null    int32 
 13  기차                501 non-null    int32 
 14  택시                501 non-null    int32 
 15  기타                501 non-null    int32 
 16  복합수단              501 non-null    int32 
dtypes: int32(12), in

In [5]:
df_ = df.drop(['자전거', '트럭', '고속시외버스', '기차', '택시', '단일 수단-계', '기타'], axis=1).loc[(df['소요시간별'] != '계') & (df['통근·통학여부별'] == '통근')].reset_index(drop=True)
df_.head(7)

,행정구역별(시군구),통근·통학여부별,소요시간별,통근·통학 인구(12세 이상),걸어서,승용차승합차(벤),시내좌석마을버스,통근통학차량,전철지하철,복합수단
0,전국,통근,15분 미만,5115688,1950898,2435638,146336,66554,35640,8
1,전국,통근,15~30,6161557,942772,3574820,701825,190527,192056,184373
2,전국,통근,30~45,7104492,306397,3784031,1283330,269914,744914,422181
3,전국,통근,45~60,1333889,17306,569859,202547,52571,304018,141634
4,전국,통근,60~90,2602371,0,1115964,249192,116824,687647,348762
5,전국,통근,90~120,926546,0,341404,65818,39661,257398,166454
6,전국,통근,120분 이상,40711,0,15756,1908,1767,6555,10204


In [6]:
# Removing unwanted values and creating '대중교통' column
values_to_remove = ['전국', '동부', '면부', '읍부']
df_ = df_[~df_['행정구역별(시군구)'].isin(values_to_remove)]
df_.reset_index(drop=True, inplace=True)
df_['대중교통'] = df_['시내좌석마을버스'] + df_['전철지하철'] + df_['복합수단']
df_.drop(columns=['시내좌석마을버스', '전철지하철', '복합수단', '통근통학차량'], inplace=True)


In [7]:
df_.head(7)

,행정구역별(시군구),통근·통학여부별,소요시간별,통근·통학 인구(12세 이상),걸어서,승용차승합차(벤),대중교통
0,서울특별시,통근,15분 미만,526138,317127,118424,41509
1,서울특별시,통근,15~30,823230,225667,232048,302550
2,서울특별시,통근,30~45,1564962,80344,441080,980969
3,서울특별시,통근,45~60,472197,5410,108803,342532
4,서울특별시,통근,60~90,846646,0,232273,580286
5,서울특별시,통근,90~120,240174,0,75141,140947
6,서울특별시,통근,120분 이상,13427,0,4288,7680


In [8]:
grouped_dfs = df_.groupby('행정구역별(시군구)')
columns_to_process = ['걸어서', '승용차승합차(벤)', '대중교통']

for group_name, group_df in grouped_dfs:
    for column in columns_to_process:
        group_df[f'accumulated_{column}'] = group_df[column].cumsum()
        group_df[f'ratio_{column}'] = group_df[f'accumulated_{column}'] / group_df[f'accumulated_{column}'].iloc[6]
        group_df[f'score_{column}'] = 1 - group_df[f'ratio_{column}']
    globals()[f"df_{group_name}"] = group_df.reset_index(drop=True)

In [9]:
regions = df_['행정구역별(시군구)'].unique()
for region in regions:
    score_df_region = globals()[f"df_{region}"][['행정구역별(시군구)', '소요시간별', '통근·통학 인구(12세 이상)', 'score_걸어서', 'score_승용차승합차(벤)', 'score_대중교통']]
    globals()[f"score_df_{region}"] = score_df_region

concatenated_df = pd.concat([globals()[f"score_df_{region}"] for region in regions], ignore_index=True)

In [10]:
from sklearn.preprocessing import MinMaxScaler

columns_of_interest = ['행정구역별(시군구)', 'score_걸어서', 'score_승용차승합차(벤)', 'score_대중교통']
df = concatenated_df[columns_of_interest]

scaler = MinMaxScaler()
scaled_data = pd.DataFrame()

for i in range(0, len(df), 7):
    subset = df.iloc[i:i+7, 1:]
    scaled_subset = scaler.fit_transform(subset)
    scaled_subset_df = pd.DataFrame(scaled_subset, columns=subset.columns)
    scaled_subset_df['행정구역별(시군구)'] = df['행정구역별(시군구)'].iloc[i:i+7].values
    scaled_data = pd.concat([scaled_data, scaled_subset_df], ignore_index=True)

scaled_data.head(7)

,score_걸어서,score_승용차승합차(벤),score_대중교통,행정구역별(시군구)
0,1.000000,1.000000,1.000000,서울특별시
1,0.275364,0.787819,0.871527,서울특별시
2,0.017372,0.384503,0.454973,서울특별시
3,0.000000,0.285015,0.309522,서울특별시
4,0.000000,0.072629,0.063112,서울특별시
5,0.000000,0.003921,0.003261,서울특별시
6,0.000000,0.000000,0.000000,서울특별시


In [11]:
final = concatenated_df[['행정구역별(시군구)', '소요시간별', '통근·통학 인구(12세 이상)']]
final = pd.concat([final, scaled_data], axis=1)
final.fillna(0, inplace=True)

print(final.head())

  행정구역별(시군구)   소요시간별  통근·통학 인구(12세 이상)  score_걸어서  score_승용차승합차(벤)  \
0      서울특별시  15분 미만            526138   1.000000         1.000000   
1      서울특별시   15~30            823230   0.275364         0.787819   
2      서울특별시   30~45           1564962   0.017372         0.384503   
3      서울특별시   45~60            472197   0.000000         0.285015   
4      서울특별시   60~90            846646   0.000000         0.072629   

   score_대중교통 행정구역별(시군구)  
0    1.000000      서울특별시  
1    0.871527      서울특별시  
2    0.454973      서울특별시  
3    0.309522      서울특별시  
4    0.063112      서울특별시  


In [12]:
# Save the region-specific DataFrames and concatenated DataFrame
# regions = df_['행정구역별(시군구)'].unique()

In [13]:
# for region in regions:
#     score_df_region = globals()[f"df_{region}"][['행정구역별(시군구)', '소요시간별', '통근·통학 인구(12세 이상)', 'score_걸어서', 'score_승용차승합차(벤)', 'score_대중교통']]
#     globals()[f"score_df_{region}"] = score_df_region

# concatenated_df = pd.concat([globals()[f"score_df_{region}"] for region in regions], ignore_index=True)

In [14]:
# Concatenate all DataFrames into a single DataFrame
# concatenated_df = pd.concat([globals()[f"score_df_{region}"] for region in regions], ignore_index=True)

In [15]:
# concatenated_df

In [16]:
# from sklearn.preprocessing import MinMaxScaler
# columns_of_interest = ['행정구역별(시군구)', 'score_걸어서', 'score_승용차승합차(벤)', 'score_대중교통']
# df = concatenated_df[columns_of_interest]
# MinMaxScaler 초기화
# scaler = MinMaxScaler()
# 결과를 저장할 데이터프레임 초기화
# scaled_data = pd.DataFrame()
# 7행씩 나누어 fit_transform 수행
# for i in range(0, len(df), 7):
#     subset = df.iloc[i:i+7, 1:]  # 'score_지역' 컬럼만 선택
#     scaled_subset = scaler.fit_transform(subset)
#     scaled_subset_df = pd.DataFrame(scaled_subset, columns=subset.columns)
#     scaled_subset_df['행정구역별(시군구)'] = df['행정구역별(시군구)'].iloc[i:i+7].values
#     scaled_data = pd.concat([scaled_data, scaled_subset_df], ignore_index=True)
# 결과 출력
# import ace_tools as tools; tools.display_dataframe_to_user(name='Scaled Data', dataframe=scaled_data)
# scaled_data.head(7)


In [17]:
# final = concatenated_df.copy()
# final = final[['행정구역별(시군구)', '소요시간별', '통근·통학 인구(12세 이상)']]
# final = pd.concat([final, scaled_data], axis = 1)
# final.isna().sum()
# print('-'*100)
# final.fillna(0, inplace= True)
# final

In [18]:
# Concatenate all DataFrames into a single DataFrame
concatenated_df = pd.concat([globals()[f"score_df_{region}"] for region in regions], ignore_index=True)

# Export the concatenated DataFrame to a CSV file
# concatenated_df.to_csv("concatenated_score_data.csv", index=False)


In [19]:
concatenated_df

,행정구역별(시군구),소요시간별,통근·통학 인구(12세 이상),score_걸어서,score_승용차승합차(벤),score_대중교통
0,서울특별시,15분 미만,526138,0.495461,0.902295,0.982679
1,서울특별시,15~30,823230,0.136432,0.710845,0.856431
2,서울특별시,30~45,1564962,0.008607,0.346935,0.447092
3,서울특별시,45~60,472197,0.000000,0.257168,0.304161
4,서울특별시,60~90,846646,0.000000,0.065532,0.062019
...,...,...,...,...,...,...
114,제주특별자치도,30~45,71244,0.001753,0.078628,0.120657
115,제주특별자치도,45~60,8414,0.000000,0.046624,0.077797
116,제주특별자치도,60~90,12559,0.000000,0.001830,0.011910
117,제주특별자치도,90~120,871,0.000000,0.000032,0.000328


In [20]:
from sklearn.preprocessing import MinMaxScaler
# 필요한 컬럼만 추출
columns_of_interest = ['행정구역별(시군구)', 'score_걸어서', 'score_승용차승합차(벤)', 'score_대중교통']
df = concatenated_df[columns_of_interest]
# MinMaxScaler 초기화
scaler = MinMaxScaler()
# 결과를 저장할 데이터프레임 초기화
scaled_data = pd.DataFrame()
# 7행씩 나누어 fit_transform 수행
for i in range(0, len(df), 7):
    subset = df.iloc[i:i+7, 1:]  # 'score_지역' 컬럼만 선택
    scaled_subset = scaler.fit_transform(subset)
    scaled_subset_df = pd.DataFrame(scaled_subset, columns=subset.columns)
    scaled_subset_df['행정구역별(시군구)'] = df['행정구역별(시군구)'].iloc[i:i+7].values
    scaled_data = pd.concat([scaled_data, scaled_subset_df], ignore_index=True)
# 결과 출력
# import ace_tools as tools; tools.display_dataframe_to_user(name="Scaled Data", dataframe=scaled_data)
scaled_data.head(7)


,score_걸어서,score_승용차승합차(벤),score_대중교통,행정구역별(시군구)
0,1.000000,1.000000,1.000000,서울특별시
1,0.275364,0.787819,0.871527,서울특별시
2,0.017372,0.384503,0.454973,서울특별시
3,0.000000,0.285015,0.309522,서울특별시
4,0.000000,0.072629,0.063112,서울특별시
5,0.000000,0.003921,0.003261,서울특별시
6,0.000000,0.000000,0.000000,서울특별시


In [21]:
#
final = concatenated_df.copy()
final = final[['행정구역별(시군구)', '소요시간별', '통근·통학 인구(12세 이상)']]
final = pd.concat([final, scaled_data], axis = 1)
# final.isna().sum()
# print('-'*100)
final.fillna(0, inplace= True)
final

,행정구역별(시군구),소요시간별,통근·통학 인구(12세 이상),score_걸어서,score_승용차승합차(벤),score_대중교통,행정구역별(시군구)
0,서울특별시,15분 미만,526138,1.000000,1.000000,1.000000,서울특별시
1,서울특별시,15~30,823230,0.275364,0.787819,0.871527,서울특별시
2,서울특별시,30~45,1564962,0.017372,0.384503,0.454973,서울특별시
3,서울특별시,45~60,472197,0.000000,0.285015,0.309522,서울특별시
4,서울특별시,60~90,846646,0.000000,0.072629,0.063112,서울특별시
...,...,...,...,...,...,...,...
114,제주특별자치도,30~45,71244,0.005650,0.116514,0.128215,제주특별자치도
115,제주특별자치도,45~60,8414,0.000000,0.069089,0.082671,제주특별자치도
116,제주특별자치도,60~90,12559,0.000000,0.002712,0.012656,제주특별자치도
117,제주특별자치도,90~120,871,0.000000,0.000047,0.000348,제주특별자치도


In [25]:
final = final.iloc[:, 0:6]
final.head(7)

,행정구역별(시군구),소요시간별,통근·통학 인구(12세 이상),score_걸어서,score_승용차승합차(벤),score_대중교통
0,서울특별시,15분 미만,526138,1.000000,1.000000,1.000000
1,서울특별시,15~30,823230,0.275364,0.787819,0.871527
2,서울특별시,30~45,1564962,0.017372,0.384503,0.454973
3,서울특별시,45~60,472197,0.000000,0.285015,0.309522
4,서울특별시,60~90,846646,0.000000,0.072629,0.063112
5,서울특별시,90~120,240174,0.000000,0.003921,0.003261
6,서울특별시,120분 이상,13427,0.000000,0.000000,0.000000


In [28]:
final.to_csv('지역별 통근시간 점수.csv', encoding='cp949')